# Optimal Feature Weights using Linear Regression

## Objective
Calculate optimal feature weights for sentiment analysis using Linear Regression.

## Goal
Solve for weights ($w$) such that:
$$\text{Price Change} \approx \sum_{i} w_i \times (\text{Feature}_{i,\text{POST}} - \text{Feature}_{i,\text{PRE}})$$

## Strategy
1. **Pivot** the sentiment dataframe so that "PRE" and "POST" values are in separate columns
2. **Calculate the Delta** (Post - Pre) for every sentiment feature
3. **Merge** the sentiment deltas with the price change data
4. **Run Linear Regression** where X = deltas, y = price_change_post
5. **Extract Coefficients** as the optimal weights

## 1. Load and Clean Data

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Load the data
df_sent = pd.read_csv("sentiment_composite.csv")
df_price = pd.read_csv("earnings_with_financials.csv")

# Keep only relevant columns for price data
columns_to_drop = [
    "symbol", "reportedDate", "fiscalDateEnding", "reportedEPS",
    "surprise", "surprisePercentage", "estimatedEPS",
    "reportTime", "price_range_prior", "price_change_prior",
    "avg_volume_prior", "price_range_post", "avg_volume_post"
]
df_price = df_price.drop(columns=columns_to_drop)

# Normalize column names for merging
df_price = df_price.rename(columns={'earnings_id': 'earningsid'})

# Remove composite_score from sentiment data (we'll build weights from scratch)
df_sent = df_sent.drop('composite_score', axis=1)

# Check for and remove null values
print("=== Checking for null values ===")
print(f"df_price nulls:\n{df_price.isnull().sum()}")
print(f"\ndf_sent nulls:\n{df_sent.isnull().sum()}")

# Remove rows with any null values
df_price_before = len(df_price)
df_sent_before = len(df_sent)

df_price = df_price.dropna()
df_sent = df_sent.dropna()

df_price_after = len(df_price)
df_sent_after = len(df_sent)

print(f"\n=== Data cleaning results ===")
print(f"df_price: removed {df_price_before - df_price_after} rows with nulls")
print(f"df_sent: removed {df_sent_before - df_sent_after} rows with nulls")
print(f"\nFinal shapes:")
print(f"df_price shape: {df_price.shape}")
print(f"df_sent shape: {df_sent.shape}")

: 

## 2. Identify Feature Columns

Extract all binary sentiment feature columns (excluding earningsid and period).

In [8]:
# Identify feature columns (exclude ID and Period)
feature_cols = [c for c in df_sent.columns if c not in ['earningsid', 'period']]

print(f"Number of features: {len(feature_cols)}")
print(f"\nFeature columns:")
for i, col in enumerate(feature_cols, 1):
    print(f"{i}. {col}")

Number of features: 15

Feature columns:
1. strong_financial_metrics
2. revenue_profit_growth
3. reasonable_valuation
4. healthy_roe
5. strong_cashflow_low_debt
6. positive_dividend_history
7. quality_management
8. competitive_advantage_moat
9. industry_leadership_and_trends
10. new_products_or_services
11. clear_positive_rating
12. risk_assessment_present
13. forward_looking_outlook
14. overall_good_stock_narrative
15. sentiment_signal


## 3. Pivot to Align PRE and POST Side-by-Side

Transform the data so that each earnings_id has PRE and POST values in separate columns.

In [9]:
# Pivot the table to align PRE and POST side-by-side
df_pivot = df_sent.pivot(index='earningsid', columns='period', values=feature_cols)

print(f"Pivot table shape: {df_pivot.shape}")
print(f"\nFirst few rows of pivoted data:")
display(df_pivot.head())

Pivot table shape: (406, 30)

First few rows of pivoted data:


strong_financial_metrics      revenue_profit_growth       \
period                         POST  PRE                  POST  PRE   
earningsid                                                            
AAPL01                          1.0  1.0                   1.0  1.0   
AAPL02                          1.0  1.0                   1.0  1.0   
AAPL03                          1.0  1.0                   1.0  1.0   
AAPL04                          1.0  1.0                   1.0  1.0   
AAPL05                          0.0  1.0                   0.0  1.0   

           reasonable_valuation      healthy_roe       \
period                     POST  PRE        POST  PRE   
earningsid                                              
AAPL01                      0.0  1.0         1.0  1.0   
AAPL02                      1.0  1.0         1.0  1.0   
AAPL03                      1.0  1.0         1.0  1.0   
AAPL04                      1.0  1.0         1.0  1.0   
AAPL05                      0.0  1.0         0.0  0.0   

           strong_cashflow_low_debt       ... clear_positive_rating       \
period                         POST  PRE  ...                  POST  PRE   
earningsid                                ...                              
AAPL01                          1.0  0.0  ...                   1.0  1.0   
AAPL02                          1.0  0.0  ...                   1.0  1.0   
AAPL03                          1.0  1.0  ...                   1.0  1.0   
AAPL04                          0.0  0.0  ...                   1.0  1.0   
AAPL05                          0.0  1.0  ...                   0.0  1.0   

           risk_assessment_present      forward_looking_outlook       \
period                        POST  PRE                    POST  PRE   
earningsid                                                             
AAPL01                         1.0  1.0                     1.0  1.0   
AAPL02                         1.0  1.0                     1.0  1.0   
AAPL03                         1.0  1.0                     1.0  1.0   
AAPL04                         1.0  1.0                     1.0  1.0   
AAPL05                         0.0  1.0                     0.0  1.0   

           overall_good_stock_narrative      sentiment_signal       
period                             POST  PRE             POST  PRE  
earningsid                                                          
AAPL01                              1.0  1.0              1.0  1.0  
AAPL02                              1.0  1.0              1.0  1.0  
AAPL03                              1.0  1.0              1.0  1.0  
AAPL04                              1.0  1.0              1.0  1.0  
AAPL05                              0.0  1.0              0.0  1.0  

[5 rows x 30 columns]

## 4. Calculate Deltas (POST - PRE)

For each feature, compute the change: POST value minus PRE value.

This represents the **change in sentiment** for each earnings event.

In [10]:
# Calculate the Delta (POST - PRE)
# The pivot creates a MultiIndex, so we access columns by (Feature, Period)
X_deltas = pd.DataFrame(index=df_pivot.index)

for col in feature_cols:
    # Post - Pre = Change in Sentiment
    X_deltas[col] = df_pivot[(col, 'POST')] - df_pivot[(col, 'PRE')]

# Check for and remove rows with NaN values (incomplete POST/PRE pairs)
rows_before = len(X_deltas)
X_deltas_clean = X_deltas.dropna()
rows_after = len(X_deltas_clean)

if rows_before > rows_after:
    print(f"Removed {rows_before - rows_after} earnings IDs with incomplete POST/PRE data")
    removed_ids = set(X_deltas.index) - set(X_deltas_clean.index)
    print(f"  IDs removed: {sorted(removed_ids)}")

# Reset index to make earningsid a column again for merging
X_deltas_clean = X_deltas_clean.reset_index()

print(f"\nDeltas shape: {X_deltas_clean.shape}")
print(f"First few rows of calculated deltas:")
display(X_deltas_clean.head())

# Show some statistics about the deltas
print(f"\nDelta statistics:")
display(X_deltas_clean[feature_cols].describe())
print(f"\nConfirm no NaN values: {X_deltas_clean[feature_cols].isnull().sum().sum()} nulls")

Removed 5 earnings IDs with incomplete POST/PRE data
  IDs removed: ['CRM15', 'MMM07', 'SHW11', 'TRV03', 'TRV15']

Deltas shape: (401, 16)
First few rows of calculated deltas:


,earningsid,strong_financial_metrics,revenue_profit_growth,reasonable_valuation,healthy_roe,strong_cashflow_low_debt,positive_dividend_history,quality_management,competitive_advantage_moat,industry_leadership_and_trends,new_products_or_services,clear_positive_rating,risk_assessment_present,forward_looking_outlook,overall_good_stock_narrative,sentiment_signal
0,AAPL01,0.0,0.0,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AAPL02,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AAPL03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,AAPL04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,AAPL05,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0



Delta statistics:


,strong_financial_metrics,revenue_profit_growth,reasonable_valuation,healthy_roe,strong_cashflow_low_debt,positive_dividend_history,quality_management,competitive_advantage_moat,industry_leadership_and_trends,new_products_or_services,clear_positive_rating,risk_assessment_present,forward_looking_outlook,overall_good_stock_narrative,sentiment_signal
count,401.000000,401.000000,401.000000,401.000000,401.000000,401.000000,401.000000,401.000000,401.000000,401.000000,401.000000,401.000000,401.000000,401.000000,401.000000
mean,0.134663,0.122195,-0.054863,0.027431,0.194514,0.124688,0.334165,0.062344,0.042394,0.054863,0.099751,0.064838,0.044888,0.029925,0.029925
std,0.454775,0.421344,0.622079,0.414422,0.653506,0.620011,0.622940,0.478125,0.375098,0.535708,0.509926,0.400981,0.279249,0.314805,0.314805
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000



Confirm no NaN values: 0 nulls


## 5. Merge Deltas with Price Change Data

Combine the feature deltas with the target variable (price_change_post).

In [11]:
# Merge with the price change target
final_df = pd.merge(X_deltas_clean, df_price, on='earningsid')

print(f"Final merged data shape: {final_df.shape}")
print(f"Number of complete observations: {len(final_df)}")
print(f"\nFirst few rows of final dataset:")
display(final_df.head())
print(f"\nConfirm no NaN values in features: {final_df[feature_cols].isnull().sum().sum()} nulls")
print(f"Confirm no NaN values in target: {final_df['price_change_post'].isnull().sum()} nulls")

Final merged data shape: (401, 17)
Number of complete observations: 401

First few rows of final dataset:


,earningsid,strong_financial_metrics,revenue_profit_growth,reasonable_valuation,healthy_roe,strong_cashflow_low_debt,positive_dividend_history,quality_management,competitive_advantage_moat,industry_leadership_and_trends,new_products_or_services,clear_positive_rating,risk_assessment_present,forward_looking_outlook,overall_good_stock_narrative,sentiment_signal,price_change_post
0,AAPL01,0.0,0.0,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.044193
1,AAPL02,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.269174
2,AAPL03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-6.040750
3,AAPL04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-10.548258
4,AAPL05,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,8.193076



Confirm no NaN values in features: 0 nulls
Confirm no NaN values in target: 0 nulls


## 6. Linear Regression to Find Optimal Weights

Fit a linear regression model where:
- **X** (Features) = The calculated deltas
- **y** (Target) = price_change_post

The coefficients will be the optimal weights for each feature.

In [12]:
# Define X (The Deltas) and y (The Price Change)
X = final_df[feature_cols]
y = final_df['price_change_post']

# Initialize Linear Regression
# fit_intercept=True allows for a baseline price drift separate from sentiment
model = LinearRegression(fit_intercept=True)

# Fit the model
model.fit(X, y)

print(f"Model fitted successfully!")
print(f"Model Intercept (Baseline Drift): {model.intercept_:.4f}")
print(f"Model R² Score: {model.score(X, y):.4f}")

Model fitted successfully!
Model Intercept (Baseline Drift): 0.0329
Model R² Score: 0.0503


## 7. Extract and Display Optimal Weights

Display the calculated weights sorted by absolute impact to see which features matter most.

In [13]:
# Extract and Display Weights
weights = pd.DataFrame({
    'Feature': feature_cols,
    'Optimal_Weight': model.coef_
})

# Sort by absolute impact to see what matters most
weights['Abs_Weight'] = weights['Optimal_Weight'].abs()
weights = weights.sort_values(by='Abs_Weight', ascending=False).drop(columns='Abs_Weight')

print("\n" + "="*60)
print("CALCULATED OPTIMAL WEIGHTS")
print("="*60)
print(f"\nModel Intercept (Baseline Drift): {model.intercept_:.4f}")
print(f"\nWeights (sorted by absolute impact):\n")
display(weights)

# Also create a horizontal bar chart for visualization
print("\nVisual representation of weight magnitudes:")
weights_sorted = weights.sort_values(by='Optimal_Weight')
print("\n(Positive weights = sentiment increase → price increase)")
print("(Negative weights = sentiment increase → price decrease)\n")
for idx, row in weights_sorted.iterrows():
    bar_length = int(abs(row['Optimal_Weight']) * 2)
    bar = '█' * bar_length if bar_length > 0 else ''
    sign = '+' if row['Optimal_Weight'] >= 0 else '-'
    print(f"{row['Feature']:40s} {sign} {bar} {row['Optimal_Weight']:7.4f}")


CALCULATED OPTIMAL WEIGHTS

Model Intercept (Baseline Drift): 0.0329

Weights (sorted by absolute impact):



,Feature,Optimal_Weight
8,industry_leadership_and_trends,2.616366
12,forward_looking_outlook,-2.508394
14,sentiment_signal,2.234334
13,overall_good_stock_narrative,2.234334
10,clear_positive_rating,-2.133735
4,strong_cashflow_low_debt,2.130483
6,quality_management,1.643597
2,reasonable_valuation,1.405688
9,new_products_or_services,-1.083253
3,healthy_roe,0.855697



Visual representation of weight magnitudes:

(Positive weights = sentiment increase → price increase)
(Negative weights = sentiment increase → price decrease)

forward_looking_outlook                  - █████ -2.5084
clear_positive_rating                    - ████ -2.1337
new_products_or_services                 - ██ -1.0833
positive_dividend_history                - █ -0.6770
competitive_advantage_moat               - █ -0.5809
risk_assessment_present                  -  -0.2397
revenue_profit_growth                    + █  0.7425
strong_financial_metrics                 + █  0.7877
healthy_roe                              + █  0.8557
reasonable_valuation                     + ██  1.4057
quality_management                       + ███  1.6436
strong_cashflow_low_debt                 + ████  2.1305
overall_good_stock_narrative             + ████  2.2343
sentiment_signal                         + ████  2.2343
industry_leadership_and_trends           + █████  2.6164


## Summary

We successfully calculated optimal weights using the pivot approach:

1. ✓ Pivoted sentiment data to separate PRE and POST columns
2. ✓ Calculated feature deltas (POST - PRE)
3. ✓ Merged deltas with price change data
4. ✓ Fitted linear regression model
5. ✓ Extracted optimal weights for each feature

The weights show how much each sentiment feature change contributes to price movement.

### Interpretation
- **Positive weights**: When this sentiment feature increases (0→1 from PRE to POST), price tends to increase
- **Negative weights**: When this sentiment feature increases, price tends to decrease
- **Large absolute values**: These features have the strongest impact on price

### Next Steps
- Consider using Ridge/Lasso regression for regularization
- Perform cross-validation to test weight stability
- Analyze residuals to check model assumptions